<a href="https://colab.research.google.com/github/obliquesignal/algo-trading-bootcamp/blob/master/5b_Convolutional_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [ ]:
from keras import models
from keras import layers

In [ ]:
#Import Python Libraries
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
plt.style.use('seaborn')

!pip install yfinance -q
import yfinance as yf 

<ipython-input-3-e9abc5df443e>:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


#Import and Process Data

In [ ]:
#Import data
start = datetime(2022, 1, 1)
end = datetime(2023, 5, 29)

stock = yf.Ticker('AAPL').history(start=start, end=end) #Apple Inc. stock
market = yf.Ticker('SPY').history(start=start, end=end) #S&P 500 index
vix = yf.Ticker('VXX').history(start=start, end=end)   #Volatility index
dxy = yf.Ticker('UUP').history(start=start, end=end)    #Dollar index
junk = yf.Ticker('JNK').history(start=start, end=end)   #Junk bond index

In [ ]:
#Create target dataframe
target = pd.DataFrame()
target['return'] = (stock['Open']-stock['Close'].shift(1))/stock['Close'].shift(1) #Returns based on buying on the close the day before and selling on the open the day after
target = target.dropna() #get rid of any NaNs
target['direction'] = np.where(target['return'] > 0, 1, 0) #Overnight direction of the stock is 1 when it is up and 0 when it is down
target.tail() #In a classification problem using binary cross-entropy, your labels have to be 0 or 1

,return,direction
Date,,
2023-05-22 00:00:00-04:00,-0.006737,0
2023-05-23 00:00:00-04:00,-0.006142,0
2023-05-24 00:00:00-04:00,-0.002740,0
2023-05-25 00:00:00-04:00,0.003317,1
2023-05-26 00:00:00-04:00,0.001908,1


In [ ]:
#Create features dataframe
features = pd.DataFrame()
features['market'] = market['Close'].pct_change(1)*100
features['vix'] = vix['Close'].diff() #Since VIX is measured in percentage terms
features['dxy'] = dxy['Close'].pct_change(1)*100
features['junk'] = junk['Close'].pct_change(1)*100
features = features.dropna()

#Values of features from the last trading session
lastknown = features[-1:]

#Subtracts last row from the features matrix so that it aligns with labels vector
features = features[:-1] 
features.tail()

,market,vix,dxy,junk
Date,,,,
2023-05-19 00:00:00-04:00,-0.145509,0.869999,-0.282187,0.054976
2023-05-22 00:00:00-04:00,0.040613,0.040001,0.070748,0.439610
2023-05-23 00:00:00-04:00,-1.122284,0.680000,0.282785,-0.492392
2023-05-24 00:00:00-04:00,-0.724480,1.469997,0.352480,-0.703760
2023-05-25 00:00:00-04:00,0.865990,-0.889999,0.456625,-0.099672


In [ ]:
#Get rid of return column for classifiers
targetclass = target.drop(axis=1, columns='return')

#Removes the first row of labels since we are correlating today's features with tomorrow's opening values
targetclass = targetclass[1:] 
targetclass.head()            

,direction
Date,
2022-01-05 00:00:00-05:00,0
2022-01-06 00:00:00-05:00,0
2022-01-07 00:00:00-05:00,1
2022-01-10 00:00:00-05:00,0
2022-01-11 00:00:00-05:00,1


In [ ]:
#Get rid of direction column for regressors
targetvalue = target.drop(axis=1, columns='direction')
targetvalue = targetvalue[1:] #Removes the first row of labels since we are correlating today's features with tomorrow's opening values
targetvalue.head()               

,return
Date,
2022-01-05 00:00:00-05:00,-0.000501
2022-01-06 00:00:00-05:00,-0.012691
2022-01-07 00:00:00-05:00,0.005174
2022-01-10 00:00:00-05:00,-0.017947
2022-01-11 00:00:00-05:00,0.000755


In [ ]:
#Standardize data
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

#Need to covert column vector into a 1-d Numpy array
targetclass = np.ravel(targetclass) 

#Every feature how have a mean of zero and standard deviation of 1
scaler = StandardScaler() 
features_standardized = scaler.fit_transform(features) 

In [ ]:
#Split dataset into train and test subsets. Test size is 25% of the total dataset and the data are not shuffled to preserve temporal structure of timeseries

features_train, features_test, targetclass_train, targetclass_test = train_test_split(features_standardized, targetclass, test_size = 0.25, shuffle=False, random_state=0)

In [ ]:
#Reshape training data because Conv1D only accepts input shape as (samples, timesteps, features)

samples = features_train.shape[0]

#Convert features into time steps
timesteps = features_train.shape[1]

#Each feature is now represented by 1 number because we are going to treat input data as a univariate time series, with only one feature per time step.
cnn_feature = 1

train_data = features_train.reshape(samples, timesteps, cnn_feature)

In [ ]:
#Reshape test data in a similar manner
test_data = features_test.reshape(features_test.shape[0], features_test.shape[1], cnn_feature)

# Develop Network

In [ ]:
#Stacking layers to create a convolutional network
cnn = models.Sequential()

#Conv1D is used since we are dealing with 1D tensors or vectors
#Filter performs the convolution operation on the input data. Kernel size is the shape of the filter matrix. 
cnn.add(layers.Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(timesteps, cnn_feature)))

#Add regularizing layer that drops half the neurons randomly and reduces overfitting
cnn.add(layers.Dropout(0.5))

cnn.add(layers.Conv1D(filters=32, kernel_size=2, activation='relu'))

#Maxpooling layers summarizes the output of all the previous layers
cnn.add(layers.MaxPooling1D(pool_size=2))

#Need to flatten the output
cnn.add(layers.Flatten())

#Add a fully connected layer that will classify the data
cnn.add(layers.Dense(units=1, activation='relu'))

#Network will minimize the binary cross entropy loss function using the Adam optimizer and will evaluate success using accuracy metric
cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')


In [ ]:
#Summarize the network
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 3, 64)             192       
                                                                 
 dropout (Dropout)           (None, 3, 64)             0         
                                                                 
 conv1d_1 (Conv1D)           (None, 2, 32)             4128      
                                                                 
 max_pooling1d (MaxPooling1D  (None, 1, 32)            0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                        

# Train and Test Network

In [ ]:
cnn.fit(train_data, targetclass_train, epochs=20);

Epoch 1/20
9/9 [==============================] - 2s 6ms/step - loss: 1.3424 - accuracy: 0.5344
Epoch 2/20
9/9 [==============================] - 0s 6ms/step - loss: 1.3216 - accuracy: 0.4771
Epoch 3/20
9/9 [==============================] - 0s 6ms/step - loss: 1.0698 - accuracy: 0.5191
Epoch 4/20
9/9 [==============================] - 0s 5ms/step - loss: 1.0788 - accuracy: 0.5153
Epoch 5/20
9/9 [==============================] - 0s 5ms/step - loss: 0.9208 - accuracy: 0.4847
Epoch 6/20
9/9 [==============================] - 0s 5ms/step - loss: 0.9724 - accuracy: 0.5153
Epoch 7/20
9/9 [==============================] - 0s 7ms/step - loss: 0.9610 - accuracy: 0.5076
Epoch 8/20
9/9 [==============================] - 0s 6ms/step - loss: 1.0778 - accuracy: 0.5305
Epoch 9/20
9/9 [==============================] - 0s 5ms/step - loss: 1.0175 - accuracy: 0.5000
Epoch 10/20
9/9 [==============================] - 0s 6ms/step - loss: 0.9440 - accuracy: 0.4771
Epoch 11/20
9/9 [======================

In [ ]:
cnn.evaluate(test_data, targetclass_test)

3/3 [==============================] - 0s 5ms/step - loss: 0.7177 - accuracy: 0.5795


[0.717738687992096, 0.5795454382896423]